<a href="https://colab.research.google.com/github/veritaem/GunHeatMapAppThingStuffAndBitsIFind/blob/master/Dataset_Exp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
!pip install plotly --upgrade


Requirement already up-to-date: plotly in /usr/local/lib/python3.6/dist-packages (3.7.0)


In [0]:
# !pip install -U git+https://github.com/matplotlib/basemap.git

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode
from matplotlib.colors import rgb2hex
from matplotlib.patches import Polygon

In [37]:
!pip install kaggle

In [38]:
from google.colab import drive
drive.mount('/content/drive')
%env KAGGLE_CONFIG_DIR=/content/drive/My Drive/

#!kaggle datasets download -d jameslko/gun-violence-data

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
env: KAGGLE_CONFIG_DIR=/content/drive/My Drive/


In [0]:
#!unzip *.zip

In [0]:
train_df = pd.read_csv('gun-violence-data_01-2013_03-2018.csv')

In [0]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [42]:
train_df.dtypes

incident_id                      int64
date                            object
state                           object
city_or_county                  object
address                         object
n_killed                         int64
n_injured                        int64
incident_url                    object
source_url                      object
incident_url_fields_missing       bool
congressional_district         float64
gun_stolen                      object
gun_type                        object
incident_characteristics        object
latitude                       float64
location_description            object
longitude                      float64
n_guns_involved                float64
notes                           object
participant_age                 object
participant_age_group           object
participant_gender              object
participant_name                object
participant_relationship        object
participant_status              object
participant_type         

In [43]:

Vegas_sht = ['sban_1', '2017-10-01', 'Nevada', 'Las Vegas', 'Mandalay Bay 3950 Blvd S', 59, 489, 'https://en.wikipedia.org/wiki/2017_Las_Vegas_shooting', 'https://en.wikipedia.org/wiki/2017_Las_Vegas_shooting', '-', '-', '-', '-', '-', '36.095', 'Hotel', 
               '-115.171667', 47, 'Route 91 Harvest Festiva; concert, open fire from 32nd floor. 47 guns seized; TOTAL:59 kill, 489 inj, number shot TBD,girlfriend Marilou Danley POI', '-', '-', '-', '-', '-', '-', '-', '-', '-', '-']
train_df.loc[len(train_df)] = Vegas_sht
train_df['date'] = pd.to_datetime(train_df['date'])
train_df['year'] = train_df['date'].dt.year
train_df['month'] = train_df['date'].dt.month
train_df['day_of_month'] = train_df['date'].dt.day
train_df['day_of_week'] = train_df['date'].dt.weekday
train_df['total'] = train_df['n_killed'] + train_df['n_injured']
df2 = train_df
train_df.head()

,incident_id,date,state,city_or_county,address,n_killed,n_injured,incident_url,source_url,incident_url_fields_missing,congressional_district,gun_stolen,gun_type,incident_characteristics,latitude,location_description,longitude,n_guns_involved,notes,participant_age,participant_age_group,participant_gender,participant_name,participant_relationship,participant_status,participant_type,sources,state_house_district,state_senate_district,year,month,day_of_month,day_of_week,total
0,461105,2013-01-01,Pennsylvania,Mckeesport,1506 Versailles Avenue and Coursin Street,0,4,http://www.gunviolencearchive.org/incident/461105,http://www.post-gazette.com/local/south/2013/0...,False,14,NaN,NaN,Shot - Wounded/Injured||Mass Shooting (4+ vict...,40.3467,NaN,-79.8559,NaN,Julian Sims under investigation: Four Shot and...,0::20,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male||1::Male||3::Male||4::Female,0::Julian Sims,NaN,0::Arrested||1::Injured||2::Injured||3::Injure...,0::Victim||1::Victim||2::Victim||3::Victim||4:...,http://pittsburgh.cbslocal.com/2013/01/01/4-pe...,NaN,NaN,2013,1,1,1,4
1,460726,2013-01-01,California,Hawthorne,13500 block of Cerise Avenue,1,3,http://www.gunviolencearchive.org/incident/460726,http://www.dailybulletin.com/article/zz/201301...,False,43,NaN,NaN,"Shot - Wounded/Injured||Shot - Dead (murder, a...",33.909,NaN,-118.333,NaN,Four Shot; One Killed; Unidentified shooter in...,0::20,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male,0::Bernard Gillis,NaN,0::Killed||1::Injured||2::Injured||3::Injured,0::Victim||1::Victim||2::Victim||3::Victim||4:...,http://losangeles.cbslocal.com/2013/01/01/man-...,62,35,2013,1,1,1,4
2,478855,2013-01-01,Ohio,Lorain,1776 East 28th Street,1,3,http://www.gunviolencearchive.org/incident/478855,http://chronicle.northcoastnow.com/2013/02/14/...,False,9,0::Unknown||1::Unknown,0::Unknown||1::Unknown,"Shot - Wounded/Injured||Shot - Dead (murder, a...",41.4455,Cotton Club,-82.1377,2.0,NaN,0::25||1::31||2::33||3::34||4::33,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male||1::Male||2::Male||3::Male||4::Male,0::Damien Bell||1::Desmen Noble||2::Herman Sea...,NaN,"0::Injured, Unharmed, Arrested||1::Unharmed, A...",0::Subject-Suspect||1::Subject-Suspect||2::Vic...,http://www.morningjournal.com/general-news/201...,56,13,2013,1,1,1,4
3,478925,2013-01-05,Colorado,Aurora,16000 block of East Ithaca Place,4,0,http://www.gunviolencearchive.org/incident/478925,http://www.dailydemocrat.com/20130106/aurora-s...,False,6,NaN,NaN,"Shot - Dead (murder, accidental, suicide)||Off...",39.6518,NaN,-104.802,NaN,NaN,0::29||1::33||2::56||3::33,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Female||1::Male||2::Male||3::Male,0::Stacie Philbrook||1::Christopher Ratliffe||...,NaN,0::Killed||1::Killed||2::Killed||3::Killed,0::Victim||1::Victim||2::Victim||3::Subject-Su...,http://denver.cbslocal.com/2013/01/06/officer-...,40,28,2013,1,5,5,4
4,478959,2013-01-07,North Carolina,Greensboro,307 Mourning Dove Terrace,2,2,http://www.gunviolencearchive.org/incident/478959,http://www.journalnow.com/news/local/article_d...,False,6,0::Unknown||1::Unknown,0::Handgun||1::Handgun,"Shot - Wounded/Injured||Shot - Dead (murder, a...",36.114,NaN,-79.9569,2.0,Two firearms recovered. (Attempted) murder sui...,0::18||1::46||2::14||3::47,0::Adult 18+||1::Adult 18+||2::Teen 12-17||3::...,0::Female||1::Male||2::Male||3::Female,0::Danielle Imani Jameison||1::Maurice Eugene ...,3::Family,0::Injured||1::Injured||2::Killed||3::Killed,0::Victim||1::Victim||2::Victim||3::Subject-Su...,http://myfox8.com/2013/01/08/update-mother-sho...,62,27,2013,1,7,0,4


In [44]:
train_df.participant_age_group.unique()

array(['0::Adult 18+||1::Adult 18+||2::Adult 18+||3::Adult 18+||4::Adult 18+',
       '0::Adult 18+||1::Adult 18+||2::Adult 18+||3::Adult 18+',
       '0::Adult 18+||1::Adult 18+||2::Teen 12-17||3::Adult 18+',
       '0::Adult 18+||1::Adult 18+||2::Adult 18+||3::Adult 18+||4::Adult 18+||5::Adult 18+',
       '0::Adult 18+||1::Adult 18+||2::Child 0-11||3::Child 0-11||4::Child 0-11||5::Teen 12-17',
       nan, '0::Teen 12-17||1::Teen 12-17||2::Teen 12-17||4::Adult 18+',
       '0::Teen 12-17||1::Adult 18+||2::Adult 18+||3::Adult 18+||4::Adult 18+||5::Adult 18+||6::Adult 18+',
       '0::Adult 18+',
       '0::Adult 18+||1::Adult 18+||2::Adult 18+||4::Adult 18+||5::Adult 18+',
       '0::Adult 18+||1::Adult 18+||2::Adult 18+||3::Adult 18+||4::Adult 18+||5::Adult 18+||6::Adult 18+',
       '0::Adult 18+||1::Adult 18+',
       '0::Adult 18+||2::Adult 18+||3::Adult 18+||4::Adult 18+||5::Adult 18+||6::Adult 18+||7::Adult 18+||8::Adult 18+',
       '0::Adult 18+||1::Adult 18+||2::Adult 18+||3:

In [0]:
#plt.figure(figsize=(12,12))
#sns.jointplot(x=train_df.latitude, y=train_df.longitude)
#plt.ylabel('Longitude', fontsize=12)
#plt.xlabel('Latitude', fontsize=12)
#plt.show()

In [46]:
Texas = train_df[train_df.state == 'Texas']
texas_hurt_count = []
for i in Texas.year.unique():
    texas_hurt_count.append(sum(Texas.n_injured[Texas.year == i]))
texas_kill_count = []
for i in Texas.year.unique():
    texas_kill_count.append(sum(Texas.n_killed[Texas.year == i]))
Texas.head()

,incident_id,date,state,city_or_county,address,n_killed,n_injured,incident_url,source_url,incident_url_fields_missing,congressional_district,gun_stolen,gun_type,incident_characteristics,latitude,location_description,longitude,n_guns_involved,notes,participant_age,participant_age_group,participant_gender,participant_name,participant_relationship,participant_status,participant_type,sources,state_house_district,state_senate_district,year,month,day_of_month,day_of_week,total
102,487891,2013-06-15,Texas,Houston,9780 Bissonnet,1,3,http://www.gunviolencearchive.org/incident/487891,http://www.houstontx.gov/police/nr/2013/jun/nr...,False,9,NaN,NaN,"Shot - Wounded/Injured||Shot - Dead (murder, a...",29.6754,Manny's Place,-95.5499,NaN,1 killed; 3 inj. Parking lot of after hrs stri...,0::43||1::24||2::27||3::28,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male||1::Male||2::Female||3::Female||4::Male,NaN,NaN,0::Killed||1::Injured||2::Injured||3::Injured|...,0::Victim||1::Victim||2::Victim||3::Victim||4:...,http://abc13.com/archive/9140359/||http://www....,146,13,2013,6,15,5,4
122,490245,2013-07-01,Texas,Fort Worth,2900 block of Pate Drive,4,0,http://www.gunviolencearchive.org/incident/490245,http://crimeblog.dallasnews.com/2013/07/man-ar...,False,33,NaN,NaN,"Shot - Dead (murder, accidental, suicide)||Mas...",32.7157,NaN,-97.2597,NaN,Unborn child killed by shot to the mother's ab...,0::39||1::22||2::10||3::0||4::22,0::Adult 18+||1::Adult 18+||2::Child 0-11||3::...,0::Female||1::Female||2::Male||4::Male,0::Annette Carroll Reed||1::Chanice Renee Reed...,4::Family,0::Killed||1::Killed||2::Killed||3::Killed||4:...,0::Victim||1::Victim||2::Victim||3::Victim||4:...,http://www.nbcdfw.com/news/local/Three-Shot-in...,95,10,2013,7,1,0,4
152,491287,2013-07-28,Texas,Dallas,2400 block of Royal Lane,0,4,http://www.gunviolencearchive.org/incident/491287,http://www.nbcdfw.com/news/local/Four-Wounded-...,False,24,NaN,NaN,Shot - Wounded/Injured||Mass Murder (4+ deceas...,32.8958,McDonald's,-96.8986,NaN,NaN,NaN,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male||1::Male||2::Male||3::Male,NaN,NaN,0::Injured||1::Injured||2::Injured||3::Injured,0::Victim||1::Victim||2::Victim||3::Victim,http://www.wfaa.com/story/news/crime/2014/08/1...,103,16,2013,7,28,6,4
163,491698,2013-08-07,Texas,Dallas,7100 block of Long Canyon Trail,4,4,http://www.gunviolencearchive.org/incident/491698,http://crimeblog.dallasnews.com/2013/08/desoto...,False,30,NaN,NaN,"Shot - Wounded/Injured||Shot - Dead (murder, a...",32.6412,NaN,-96.9481,NaN,"Address #1 as shown above, address #2 is 141 G...",0::47||1::28||2::11||3::13||4::43||5::17||6::1...,0::Adult 18+||1::Adult 18+||2::Child 0-11||3::...,0::Female||1::Female||2::Male||3::Male||4::Fem...,0::Zina Bowser||1::Neima Williams||2::Myles||3...,8::Significant others - current or former,0::Killed||1::Killed||2::Injured||3::Injured||...,0::Victim||1::Victim||2::Victim||3::Victim||4:...,http://www.dallasnews.com/news/crime/headlines...,111,23,2013,8,7,2,8
202,480605,2013-09-20,Texas,Rice,700 block of NW 0149,5,0,http://www.gunviolencearchive.org/incident/480605,http://www.corsicanadailysun.com/news/local_ne...,False,6,0::Unknown,0::22 LR,"Shot - Dead (murder, accidental, suicide)||Sui...",32.2337,NaN,-96.4661,1.0,NaN,0::33||1::10||2::8||3::4||4::33,0::Adult 18+||1::Child 0-11||2::Child 0-11||3:...,0::Male||1::Male||2::Male||3::Male||4::Female,0::Israel Chairez Alvarez||1::Misael Alvarez||...,4::Family,0::Killed||1::Killed||2::Killed||3::Killed||4:...,0::Victim||1::Victim||2::Victim||3::Victim||4:...,http://www.nbcdfw.com/news/local/Five-Family-M...,NaN,22,2013,9,20,4,5


In [61]:
trace1 = go.Scatter(
                    x = np.array(["2013", "2014", "2015","2016","2017", "2018"]),
                    y = texas_kill_count,
                    mode = "lines+markers",
                    name = "killed",
                    marker = dict(color = 'rgba(80, 26, 80, 0.8)'),
                    text= "")

data = [trace1]
layout = dict(title = 'Number of total ppl Killed in Texas between 2014 and 2017',
              xaxis= dict(title= 'Year',ticklen= 5,zeroline= False),
              yaxis= dict(title= 'Gun Violence Number',ticklen= 5,zeroline= False)
             )
fig = dict(data = data, layout = layout)
plotly.offline.plot(fig, "testtime.html")

'file:///content/temp-plot.html'

In [0]:
total_kill_count = []
for i in Texas.year.unique():
    texas_kill_count.append(sum(Texas.n_killed[Texas.year == i])

In [101]:
temp = train_df.groupby('date').agg({'state' : 'count', 'n_killed' : 'sum', 'n_injured' : 'sum'}).reset_index().rename(columns={'state' : 'incidents'})


trace1 = go.Scatter(x = temp.date, y = temp.incidents, name='Total Incidents', mode = "lines", marker = dict(color = '#c5d9f9'))
trace2 = go.Scatter(x = temp.date, y = temp.n_killed, name="Total Killed", mode = "lines", marker = dict(color = '#ff9f87'))
trace3 = go.Scatter(x = temp.date, y = temp.n_injured, name="Total Injured", mode = "lines", marker = dict(color = '#e8baff'))

data = [trace1, trace2, trace3]
layout = dict(height=350,title = 'Gun Violence Incidents - 2017', legend=dict(orientation="h", x=-.01, y=1), xaxis= dict(title='Date Time', ticklen= 1))
fig = dict(data = data, layout = layout)
plotly.offline.plot(fig, "fig.html")

'file:///content/temp-plot.html'

In [62]:
trace1 = go.Scatter(
                    x = np.array(["2014","2015","2016","2017", "2018"]),
                    y = texas_hurt_count,
                    mode = "lines+markers",
                    name = "injured",
                    marker = dict(color = 'rgba(80, 26, 80, 0.8)'),
                    text= "")

data = [trace1]
layout = dict(title = 'Number of total ppl Injured in Texas between 2014 and 2017',
              xaxis= dict(title= 'Year',ticklen= 5,zeroline= False),
              yaxis= dict(title= 'Gun Violence Number',ticklen= 5,zeroline= False)
             )
fig = dict(data = data, layout = layout)
plotly.offline.plot(fig, "testtime.html")

'file:///content/temp-plot.html'

In [0]:
trace1 = go.Scatter(
                    x = np.array(["2014","2015","2016","2017", "2018"]),
                    y = texas_hurt_count,
                    mode = "lines+markers",
                    name = "injured",
                    marker = dict(color = 'rgba(80, 26, 80, 0.8)'),
                    text= "")

data = [trace1]
layout = dict(title = 'Number of total ppl Injured in Texas between 2014 and 2017',
              xaxis= dict(title= 'Year',ticklen= 5,zeroline= False),
              yaxis= dict(title= 'Gun Violence Number',ticklen= 5,zeroline= False)
             )
fig = dict(data = data, layout = layout)
plotly.offline.plot(fig, "testtime.html")

In [56]:
texas = train_df[train_df.state == "Texas"]
texas[texas.year == 2014].n_injured.sum()

1157

In [0]:
train_df['loss'] = train_df['n_killed'] + train_df['n_injured']

In [0]:
from plotly.offline import init_notebook_mode, iplot
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from wordcloud import WordCloud
from textblob import TextBlob 
import plotly.plotly as py
from plotly import tools
import seaborn as sns
import pandas as pd
import string, os, random
import calendar
from PIL import Image 
import numpy as np

import folium 
from folium import plugins 

In [98]:
smalldf[smalldf['loss'] >= 5]

,latitude,longitude,loss,n_killed,state,city_or_county
6,34.9791,-106.716,5,5,New Mexico,Albuquerque
20,39.7407,-75.5499,5,3,Delaware,Wilmington
21,40.6008,-111.903,5,4,Utah,Midvale
22,33.8031,-117.943,7,4,California,Orange (county)
36,43.011,-75.0058,8,6,New York,Mohawk
44,37.3603,-120.654,5,3,California,Merced (county)
45,47.3149,-122.224,5,3,Washington,Auburn
54,47.303,-122.31,5,5,Washington,Federal Way
57,39.5417,-90.3301,7,6,Illinois,Manchester
69,39.1149,-85.8929,5,5,Indiana,Columbus


In [96]:
smalldf = train_df[train_df['n_killed'] >= 3][['latitude', 'longitude', 'loss', 'n_killed', 'state', 'city_or_county']].dropna()
map1 = folium.Map(location=[39.50, -98.35], tiles='CartoDB dark_matter', zoom_start=3.5)
map2 = folium.Map([39.50, -98.35], zoom_start=3.5, tiles='cartodbdark_matter')

markers = []
for i, row in smalldf.iterrows():
    loss = row['loss']
    if row['loss'] > 5:
        loss = row['loss']*0.1    
        folium.CircleMarker([float(row['latitude']), float(row['longitude'])],
                            radius=float(loss * 5), color='#ef4f61').add_to(map1)
map1

In [72]:
Texas['incident_type'] = Texas.incident_characteristics.str.replace(r'((?i)^.*mass.*$)', 'Mass Shooting')
Texas['incident_type'] = Texas.incident_characteristics.str.replace(r'((?i)^.*murder.*$)', 'Homicide')
Texas['incident_type'] = Texas.incident_characteristics.str.replace(r'((?i)^.*police.*$)', 'Police Involved')
Texas['incident_type'] = Texas.incident_characteristics.str.replace(r'((?i)^.*officer.*$)', 'Police Involved')
Texas['incident_type'] = Texas.incident_characteristics.str.replace(r'((?i)^.*accident.*$)', 'Accidental')
Texas['incident_type'] = Texas.incident_characteristics.str.replace(r'((?i)^.*accidental.*$)', 'Accidental')
Texas['incident_type'] = Texas.incident_characteristics.str.replace(r'((?i)^.*child.*$)', 'Child Involved')
Texas['incident_type'] = Texas.incident_characteristics.str.replace(r'((?i)^.*suicide.*$)', 'Suicide')
Texas['incident_type'] = Texas.incident_characteristics.str.replace(r'((?i)^.*raid.*$)', 'Raid')


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-ve

In [12]:
for col in train_df.columns:
    train_df[col] = train_df[col].astype(str)


train_df['text'] = train_df['state'] + '<br>' + \
    'number of deaths' + train_df['n_killed'] + '<br>' + \
    'number injured ' + train_df['n_injured'] + '<br>' + \
    'where ' + train_df['city_or_county'] + '<br>' + \
    ' When ' + train_df['date']



train_df.head()


,incident_id,date,state,city_or_county,address,n_killed,n_injured,incident_url,source_url,incident_url_fields_missing,congressional_district,gun_stolen,gun_type,incident_characteristics,latitude,location_description,longitude,n_guns_involved,notes,participant_age,participant_age_group,participant_gender,participant_name,participant_relationship,participant_status,participant_type,sources,state_house_district,state_senate_district,year,month,day_of_month,day_of_week,total,text
0,461105,2013-01-01,Pennsylvania,Mckeesport,1506 Versailles Avenue and Coursin Street,0,4,http://www.gunviolencearchive.org/incident/461105,http://www.post-gazette.com/local/south/2013/0...,False,14.0,nan,nan,Shot - Wounded/Injured||Mass Shooting (4+ vict...,40.3467,nan,-79.8559,nan,Julian Sims under investigation: Four Shot and...,0::20,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male||1::Male||3::Male||4::Female,0::Julian Sims,nan,0::Arrested||1::Injured||2::Injured||3::Injure...,0::Victim||1::Victim||2::Victim||3::Victim||4:...,http://pittsburgh.cbslocal.com/2013/01/01/4-pe...,nan,nan,2013,1,1,1,4,Pennsylvania<br>number of deaths0<br>number in...
1,460726,2013-01-01,California,Hawthorne,13500 block of Cerise Avenue,1,3,http://www.gunviolencearchive.org/incident/460726,http://www.dailybulletin.com/article/zz/201301...,False,43.0,nan,nan,"Shot - Wounded/Injured||Shot - Dead (murder, a...",33.909,nan,-118.333,nan,Four Shot; One Killed; Unidentified shooter in...,0::20,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male,0::Bernard Gillis,nan,0::Killed||1::Injured||2::Injured||3::Injured,0::Victim||1::Victim||2::Victim||3::Victim||4:...,http://losangeles.cbslocal.com/2013/01/01/man-...,62.0,35.0,2013,1,1,1,4,California<br>number of deaths1<br>number inju...
2,478855,2013-01-01,Ohio,Lorain,1776 East 28th Street,1,3,http://www.gunviolencearchive.org/incident/478855,http://chronicle.northcoastnow.com/2013/02/14/...,False,9.0,0::Unknown||1::Unknown,0::Unknown||1::Unknown,"Shot - Wounded/Injured||Shot - Dead (murder, a...",41.4455,Cotton Club,-82.1377,2.0,nan,0::25||1::31||2::33||3::34||4::33,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male||1::Male||2::Male||3::Male||4::Male,0::Damien Bell||1::Desmen Noble||2::Herman Sea...,nan,"0::Injured, Unharmed, Arrested||1::Unharmed, A...",0::Subject-Suspect||1::Subject-Suspect||2::Vic...,http://www.morningjournal.com/general-news/201...,56.0,13.0,2013,1,1,1,4,Ohio<br>number of deaths1<br>number injured 3<...
3,478925,2013-01-05,Colorado,Aurora,16000 block of East Ithaca Place,4,0,http://www.gunviolencearchive.org/incident/478925,http://www.dailydemocrat.com/20130106/aurora-s...,False,6.0,nan,nan,"Shot - Dead (murder, accidental, suicide)||Off...",39.6518,nan,-104.802,nan,nan,0::29||1::33||2::56||3::33,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Female||1::Male||2::Male||3::Male,0::Stacie Philbrook||1::Christopher Ratliffe||...,nan,0::Killed||1::Killed||2::Killed||3::Killed,0::Victim||1::Victim||2::Victim||3::Subject-Su...,http://denver.cbslocal.com/2013/01/06/officer-...,40.0,28.0,2013,1,5,5,4,Colorado<br>number of deaths4<br>number injure...
4,478959,2013-01-07,North Carolina,Greensboro,307 Mourning Dove Terrace,2,2,http://www.gunviolencearchive.org/incident/478959,http://www.journalnow.com/news/local/article_d...,False,6.0,0::Unknown||1::Unknown,0::Handgun||1::Handgun,"Shot - Wounded/Injured||Shot - Dead (murder, a...",36.114000000000004,nan,-79.9569,2.0,Two firearms recovered. (Attempted) murder sui...,0::18||1::46||2::14||3::47,0::Adult 18+||1::Adult 18+||2::Teen 12-17||3::...,0::Female||1::Male||2::Male||3::Female,0::Danielle Imani Jameison||1::Maurice Eugene ...,3::Family,0::Injured||1::Injured||2::Killed||3::Killed,0::Victim||1::Victim||2::Victim||3::Subject-Su...,http://myfox8.com/2013/01/08/update-mother-sho...,62.0,27.0,2013,1,7,0,4,North Carolina<br>number of deaths2<br>number ...


In [13]:
train_df.incident_characteristics.nunique()

18128

In [14]:
train_df.congressional_district.unique()

array(['14.0', '43.0', '9.0', '6.0', '1.0', '2.0', '7.0', '3.0', '8.0',
       '5.0', '4.0', '46.0', '25.0', '16.0', '49.0', '22.0', '10.0',
       '20.0', '47.0', '13.0', '18.0', '12.0', '23.0', '33.0', '17.0',
       '24.0', '30.0', 'nan', '11.0', '44.0', '26.0', '21.0', '41.0',
       '36.0', '32.0', '15.0', '19.0', '27.0', '28.0', '34.0', '31.0',
       '48.0', '35.0', '51.0', '29.0', '38.0', '39.0', '52.0', '42.0',
       '40.0', '53.0', '37.0', '50.0', '45.0', '0.0', '-'], dtype=object)

In [15]:
train_df.isna().sum()

incident_id                    0
date                           0
state                          0
city_or_county                 0
address                        0
n_killed                       0
n_injured                      0
incident_url                   0
source_url                     0
incident_url_fields_missing    0
congressional_district         0
gun_stolen                     0
gun_type                       0
incident_characteristics       0
latitude                       0
location_description           0
longitude                      0
n_guns_involved                0
notes                          0
participant_age                0
participant_age_group          0
participant_gender             0
participant_name               0
participant_relationship       0
participant_status             0
participant_type               0
sources                        0
state_house_district           0
state_senate_district          0
year                           0
month     

In [16]:
train_df.shape

(239678, 35)

In [17]:

# Creates a heatmap of gun violence in nation by state, offline to avoid plotly charges


plotly.offline.init_notebook_mode(connected=True)

states_df = train_df['state'].value_counts()

statesdf = pd.DataFrame()
statesdf['state'] = states_df.index
statesdf['counts'] = states_df.values

scl = [[0.0, 'rgb(242,240,247)'],[0.2, 'rgb(218,218,235)'],[0.4, 'rgb(188,189,220)'],\
            [0.6, 'rgb(158,154,200)'],[0.8, 'rgb(117,107,177)'],[1.0, 'rgb(84,39,143)']]

state_to_code = {'District of Columbia' : 'dc','Mississippi': 'MS', 'Oklahoma': 'OK', 'Delaware': 'DE', 'Minnesota': 'MN', 'Illinois': 'IL', 'Arkansas': 'AR', 'New Mexico': 'NM', 'Indiana': 'IN', 'Maryland': 'MD', 'Louisiana': 'LA', 'Idaho': 'ID', 'Wyoming': 'WY', 'Tennessee': 'TN', 'Arizona': 'AZ', 'Iowa': 'IA', 'Michigan': 'MI', 'Kansas': 'KS', 'Utah': 'UT', 'Virginia': 'VA', 'Oregon': 'OR', 'Connecticut': 'CT', 'Montana': 'MT', 'California': 'CA', 'Massachusetts': 'MA', 'West Virginia': 'WV', 'South Carolina': 'SC', 'New Hampshire': 'NH', 'Wisconsin': 'WI', 'Vermont': 'VT', 'Georgia': 'GA', 'North Dakota': 'ND', 'Pennsylvania': 'PA', 'Florida': 'FL', 'Alaska': 'AK', 'Kentucky': 'KY', 'Hawaii': 'HI', 'Nebraska': 'NE', 'Missouri': 'MO', 'Ohio': 'OH', 'Alabama': 'AL', 'Rhode Island': 'RI', 'South Dakota': 'SD', 'Colorado': 'CO', 'New Jersey': 'NJ', 'Washington': 'WA', 'North Carolina': 'NC', 'New York': 'NY', 'Texas': 'TX', 'Nevada': 'NV', 'Maine': 'ME'}
statesdf['state_code'] = statesdf['state'].apply(lambda x : state_to_code[x])

data = [ dict(
        type='choropleth',
        colorscale = 'rainbow',
        autocolorscale = False,
        locations = statesdf['state_code'],
        z = statesdf['counts'],
        locationmode = 'USA-states',
        text = statesdf['state'],
        marker = dict(
            line = dict (
                color = 'rgb(255,255,255)',
                width = 2
            ) ),
        colorbar = dict(
            title = "Gun Violence Incidents")
        ) ]

layout = dict(
        title = 'State wise number of Gun Violence Incidents',
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showlakes = True,
            lakecolor = 'rgb(255, 255, 255)'),
             )

    
fig = dict( data=data, layout=layout )
plotly.offline.plot( fig, filename='d3-cloropleth-map.html' )

'file:///content/d3-cloropleth-map.html'

In [18]:
# creates a bar chart showing gun crimes per 10,000 people per state, offline

census_2013 = {'Mississippi': 2991207, 'Iowa': 3090416, 'Oklahoma': 3850568, 'Delaware': 925749, 'Minnesota': 5420380, 'Alaska': 735132, 'Illinois': 12882135, 'Arkansas': 2959373, 'New Mexico': 2085287, 'Indiana': 6570902, 'Maryland': 5928814, 'Louisiana': 4625470, 'Texas': 26448193, 'Wyoming': 582658, 'Arizona': 6626624, 'Wisconsin': 5742713, 'Michigan': 9895622, 'Kansas': 2893957, 'Utah': 2900872, 'Virginia': 8260405, 'Oregon': 3930065, 'Connecticut': 3596080, 'New York': 19651127, 'California': 38332521, 'Massachusetts': 6692824, 'West Virginia': 1854304, 'South Carolina': 4774839, 'New Hampshire': 1323459, 'Vermont': 626630, 'Georgia': 9992167, 'North Dakota': 723393, 'Pennsylvania': 12773801, 'Florida': 19552860, 'Hawaii': 1404054, 'Kentucky': 4395295, 'Rhode Island': 1051511, 'Nebraska': 1868516, 'Missouri': 6044171, 'Ohio': 11570808, 'Alabama': 4833722, 'South Dakota': 844877, 'Colorado': 5268367, 'Idaho': 1612136, 'New Jersey': 8899339, 'Washington': 6971406, 'North Carolina': 9848060, 'Tennessee': 6495978, 'Montana': 1015165, 'District of Columbia': 646449, 'Nevada': 2790136, 'Maine': 1328302}
statesdf['population'] = statesdf['state'].apply(lambda x : census_2013[x])
statesdf['incidents_per_k'] = statesdf.apply(lambda x : float(x['counts'])*10000 / x['population'], axis=1)
tempdf = statesdf.sort_values('incidents_per_k', ascending = False)[1:50]
trace1 = go.Bar(
    x=tempdf.state,
    y=tempdf.incidents_per_k,
    name='Location Types',
    orientation = 'v',
    marker=dict(color='#A51212'),
    opacity=0.7
)

data = [trace1]
layout = go.Layout(
    height=400,
    margin=dict(b=150),
    barmode='group',
    legend=dict(dict(x=-.1, y=1.2)),
    title = 'Gun Crime Per 10,000 people',
)

fig = go.Figure(data=data, layout=layout)
plotly.offline.plot(fig, filename='grouped-bar.html')

'file:///content/grouped-bar.html'

In [19]:
train_df.head()

,incident_id,date,state,city_or_county,address,n_killed,n_injured,incident_url,source_url,incident_url_fields_missing,congressional_district,gun_stolen,gun_type,incident_characteristics,latitude,location_description,longitude,n_guns_involved,notes,participant_age,participant_age_group,participant_gender,participant_name,participant_relationship,participant_status,participant_type,sources,state_house_district,state_senate_district,year,month,day_of_month,day_of_week,total,text
0,461105,2013-01-01,Pennsylvania,Mckeesport,1506 Versailles Avenue and Coursin Street,0,4,http://www.gunviolencearchive.org/incident/461105,http://www.post-gazette.com/local/south/2013/0...,False,14.0,nan,nan,Shot - Wounded/Injured||Mass Shooting (4+ vict...,40.3467,nan,-79.8559,nan,Julian Sims under investigation: Four Shot and...,0::20,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male||1::Male||3::Male||4::Female,0::Julian Sims,nan,0::Arrested||1::Injured||2::Injured||3::Injure...,0::Victim||1::Victim||2::Victim||3::Victim||4:...,http://pittsburgh.cbslocal.com/2013/01/01/4-pe...,nan,nan,2013,1,1,1,4,Pennsylvania<br>number of deaths0<br>number in...
1,460726,2013-01-01,California,Hawthorne,13500 block of Cerise Avenue,1,3,http://www.gunviolencearchive.org/incident/460726,http://www.dailybulletin.com/article/zz/201301...,False,43.0,nan,nan,"Shot - Wounded/Injured||Shot - Dead (murder, a...",33.909,nan,-118.333,nan,Four Shot; One Killed; Unidentified shooter in...,0::20,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male,0::Bernard Gillis,nan,0::Killed||1::Injured||2::Injured||3::Injured,0::Victim||1::Victim||2::Victim||3::Victim||4:...,http://losangeles.cbslocal.com/2013/01/01/man-...,62.0,35.0,2013,1,1,1,4,California<br>number of deaths1<br>number inju...
2,478855,2013-01-01,Ohio,Lorain,1776 East 28th Street,1,3,http://www.gunviolencearchive.org/incident/478855,http://chronicle.northcoastnow.com/2013/02/14/...,False,9.0,0::Unknown||1::Unknown,0::Unknown||1::Unknown,"Shot - Wounded/Injured||Shot - Dead (murder, a...",41.4455,Cotton Club,-82.1377,2.0,nan,0::25||1::31||2::33||3::34||4::33,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male||1::Male||2::Male||3::Male||4::Male,0::Damien Bell||1::Desmen Noble||2::Herman Sea...,nan,"0::Injured, Unharmed, Arrested||1::Unharmed, A...",0::Subject-Suspect||1::Subject-Suspect||2::Vic...,http://www.morningjournal.com/general-news/201...,56.0,13.0,2013,1,1,1,4,Ohio<br>number of deaths1<br>number injured 3<...
3,478925,2013-01-05,Colorado,Aurora,16000 block of East Ithaca Place,4,0,http://www.gunviolencearchive.org/incident/478925,http://www.dailydemocrat.com/20130106/aurora-s...,False,6.0,nan,nan,"Shot - Dead (murder, accidental, suicide)||Off...",39.6518,nan,-104.802,nan,nan,0::29||1::33||2::56||3::33,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Female||1::Male||2::Male||3::Male,0::Stacie Philbrook||1::Christopher Ratliffe||...,nan,0::Killed||1::Killed||2::Killed||3::Killed,0::Victim||1::Victim||2::Victim||3::Subject-Su...,http://denver.cbslocal.com/2013/01/06/officer-...,40.0,28.0,2013,1,5,5,4,Colorado<br>number of deaths4<br>number injure...
4,478959,2013-01-07,North Carolina,Greensboro,307 Mourning Dove Terrace,2,2,http://www.gunviolencearchive.org/incident/478959,http://www.journalnow.com/news/local/article_d...,False,6.0,0::Unknown||1::Unknown,0::Handgun||1::Handgun,"Shot - Wounded/Injured||Shot - Dead (murder, a...",36.114000000000004,nan,-79.9569,2.0,Two firearms recovered. (Attempted) murder sui...,0::18||1::46||2::14||3::47,0::Adult 18+||1::Adult 18+||2::Teen 12-17||3::...,0::Female||1::Male||2::Male||3::Female,0::Danielle Imani Jameison||1::Maurice Eugene ...,3::Family,0::Injured||1::Injured||2::Killed||3::Killed,0::Victim||1::Victim||2::Victim||3::Subject-Su...,http://myfox8.com/2013/01/08/update-mother-sho...,62.0,27.0,2013,1,7,0,4,North Carolina<br>number of deaths2<br>number ...


In [20]:
statesdf.head()

,state,counts,state_code,population,incidents_per_k
0,Illinois,17556,IL,12882135,13.628176
1,California,16306,CA,38332521,4.253829
2,Florida,15029,FL,19552860,7.686344
3,Texas,13577,TX,26448193,5.133432
4,Ohio,10244,OH,11570808,8.853314


In [21]:


scl = [[0.0, 'rgb(242,240,247)'],[0.2, 'rgb(218,218,235)'],[0.4, 'rgb(188,189,220)'],\
            [0.6, 'rgb(158,154,200)'],[0.8, 'rgb(117,107,177)'],[1.0, 'rgb(84,39,143)']]

data = [ dict(
        type='choropleth',
        colorscale = scl,
        autocolorscale = True,
        locations = statesdf['state_code'],
        z = statesdf['incidents_per_k'],
        locationmode = 'USA-states',
        text = statesdf['state'],
        marker = dict(
            line = dict (
                color = 'rgb(255,255,255)',
                width = 2
            ) ),
       
        colorbar = dict(
            title = "Gun Violence Incidents")
        ) ]

layout = dict(
        title = 'State wise number of Gun Violence Incidents',
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showlakes = True,
            lakecolor = 'rgb(255, 255, 255)'),
             )

    
fig = dict( data=data, layout=layout )
plotly.offline.plot( fig, filename='d3-cloropleth-map.html' )

'file:///content/d3-cloropleth-map.html'

In [22]:
statesdf.head()

,state,counts,state_code,population,incidents_per_k
0,Illinois,17556,IL,12882135,13.628176
1,California,16306,CA,38332521,4.253829
2,Florida,15029,FL,19552860,7.686344
3,Texas,13577,TX,26448193,5.133432
4,Ohio,10244,OH,11570808,8.853314


In [0]:
statesdf['year'] = train_df['year']
statesdf['date'] = train_df['date']

In [0]:
train_df['incident_type'] = train_df.incident_characteristics.str.replace(r'((?i)^.*mass.*$)', 'Mass Shooting')
train_df['incident_type'] = train_df.incident_characteristics.str.replace(r'((?i)^.*murder.*$)', 'Homicide')
train_df['incident_type'] = train_df.incident_characteristics.str.replace(r'((?i)^.*police.*$)', 'Police Involved')
train_df['incident_type'] = train_df.incident_characteristics.str.replace(r'((?i)^.*officer.*$)', 'Police Involved')
train_df['incident_type'] = train_df.incident_characteristics.str.replace(r'((?i)^.*accident.*$)', 'Accidental')
train_df['incident_type'] = train_df.incident_characteristics.str.replace(r'((?i)^.*child.*$)', 'Child Involved')
train_df['incident_type'] = train_df.incident_characteristics.str.replace(r'((?i)^.*suicide.*$)', 'Suicide')


In [26]:
temp = train_df[train_df['year'] == 2014].groupby('date').agg({'state' : 'count', 'n_killed' : 'sum', 'n_injured' : 'sum'}).reset_index().rename(columns={'state' : 'incidents'})


trace1 = go.Scatter(x = temp.date, y = temp.incidents, name='Total Incidents', mode = "lines", marker = dict(color = '#c5d9f9'))
trace2 = go.Scatter(x = temp.date, y = temp.n_killed, name="Total Killed", mode = "lines", marker = dict(color = '#ff9f87'))
trace3 = go.Scatter(x = temp.date, y = temp.n_injured, name="Total Injured", mode = "lines", marker = dict(color = '#e8baff'))

data = [trace1, trace2, trace3]
layout = dict(height=350, title = 'Gun Violence Incidents - 2014', legend=dict(orientation="h", x=-.01, y=1), xaxis= dict(title='Date Time', ticklen= 1))
fig = dict(data = data, layout = layout)
plotly.offline.plot(fig, 'timeseries.html')

'file:///content/temp-plot.html'

In [0]:
temp = df2[df2['year'] == 2016].groupby('date').agg({'state' : 'count', 'n_killed' : 'sum', 'n_injured' : 'sum'}).reset_index().rename(columns={'state' : 'incidents'})


trace1 = go.Scatter(x = temp.date, y = temp.incidents, name='Total Incidents', mode = "lines", marker = dict(color = '#c5d9f9'))
trace2 = go.Scatter(x = temp.date, y = temp.n_killed, name="Total Killed", mode = "lines", marker = dict(color = '#ff9f87'))
trace3 = go.Scatter(x = temp.date, y = temp.n_injured, name="Total Injured", mode = "lines", marker = dict(color = '#e8baff'))

data = [trace1, trace2, trace3]
layout = dict(height=350,title = 'Gun Violence Incidents - 2017', legend=dict(orientation="h", x=-.01, y=1), xaxis= dict(title='Date Time', ticklen= 1))
fig = dict(data = data, layout = layout)
plotly.offline.plot(fig, filename='timechart.html')

In [70]:
Texas.head()

,incident_id,date,state,city_or_county,address,n_killed,n_injured,incident_url,source_url,incident_url_fields_missing,congressional_district,gun_stolen,gun_type,incident_characteristics,latitude,location_description,longitude,n_guns_involved,notes,participant_age,participant_age_group,participant_gender,participant_name,participant_relationship,participant_status,participant_type,sources,state_house_district,state_senate_district,year,month,day_of_month,day_of_week,total,incident_type
102,487891,2013-06-15,Texas,Houston,9780 Bissonnet,1,3,http://www.gunviolencearchive.org/incident/487891,http://www.houstontx.gov/police/nr/2013/jun/nr...,False,9,NaN,NaN,"Shot - Wounded/Injured||Shot - Dead (murder, a...",29.6754,Manny's Place,-95.5499,NaN,1 killed; 3 inj. Parking lot of after hrs stri...,0::43||1::24||2::27||3::28,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male||1::Male||2::Female||3::Female||4::Male,NaN,NaN,0::Killed||1::Injured||2::Injured||3::Injured|...,0::Victim||1::Victim||2::Victim||3::Victim||4:...,http://abc13.com/archive/9140359/||http://www....,146,13,2013,6,15,5,4,Suicide
122,490245,2013-07-01,Texas,Fort Worth,2900 block of Pate Drive,4,0,http://www.gunviolencearchive.org/incident/490245,http://crimeblog.dallasnews.com/2013/07/man-ar...,False,33,NaN,NaN,"Shot - Dead (murder, accidental, suicide)||Mas...",32.7157,NaN,-97.2597,NaN,Unborn child killed by shot to the mother's ab...,0::39||1::22||2::10||3::0||4::22,0::Adult 18+||1::Adult 18+||2::Child 0-11||3::...,0::Female||1::Female||2::Male||4::Male,0::Annette Carroll Reed||1::Chanice Renee Reed...,4::Family,0::Killed||1::Killed||2::Killed||3::Killed||4:...,0::Victim||1::Victim||2::Victim||3::Victim||4:...,http://www.nbcdfw.com/news/local/Three-Shot-in...,95,10,2013,7,1,0,4,Suicide
152,491287,2013-07-28,Texas,Dallas,2400 block of Royal Lane,0,4,http://www.gunviolencearchive.org/incident/491287,http://www.nbcdfw.com/news/local/Four-Wounded-...,False,24,NaN,NaN,Shot - Wounded/Injured||Mass Murder (4+ deceas...,32.8958,McDonald's,-96.8986,NaN,NaN,NaN,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male||1::Male||2::Male||3::Male,NaN,NaN,0::Injured||1::Injured||2::Injured||3::Injured,0::Victim||1::Victim||2::Victim||3::Victim,http://www.wfaa.com/story/news/crime/2014/08/1...,103,16,2013,7,28,6,4,Shot - Wounded/Injured||Mass Murder (4+ deceas...
163,491698,2013-08-07,Texas,Dallas,7100 block of Long Canyon Trail,4,4,http://www.gunviolencearchive.org/incident/491698,http://crimeblog.dallasnews.com/2013/08/desoto...,False,30,NaN,NaN,"Shot - Wounded/Injured||Shot - Dead (murder, a...",32.6412,NaN,-96.9481,NaN,"Address #1 as shown above, address #2 is 141 G...",0::47||1::28||2::11||3::13||4::43||5::17||6::1...,0::Adult 18+||1::Adult 18+||2::Child 0-11||3::...,0::Female||1::Female||2::Male||3::Male||4::Fem...,0::Zina Bowser||1::Neima Williams||2::Myles||3...,8::Significant others - current or former,0::Killed||1::Killed||2::Injured||3::Injured||...,0::Victim||1::Victim||2::Victim||3::Victim||4:...,http://www.dallasnews.com/news/crime/headlines...,111,23,2013,8,7,2,8,Suicide
202,480605,2013-09-20,Texas,Rice,700 block of NW 0149,5,0,http://www.gunviolencearchive.org/incident/480605,http://www.corsicanadailysun.com/news/local_ne...,False,6,0::Unknown,0::22 LR,"Shot - Dead (murder, accidental, suicide)||Sui...",32.2337,NaN,-96.4661,1.0,NaN,0::33||1::10||2::8||3::4||4::33,0::Adult 18+||1::Child 0-11||2::Child 0-11||3:...,0::Male||1::Male||2::Male||3::Male||4::Female,0::Israel Chairez Alvarez||1::Misael Alvarez||...,4::Family,0::Killed||1::Killed||2::Killed||3::Killed||4:...,0::Victim||1::Victim||2::Victim||3::Victim||4:...,http://www.nbcdfw.com/news/local/Five-Family-M...,NaN,22,2013,9,20,4,5,Suicide


In [31]:
trace1 = go.Scatter(
                    x = np.array(["2014","2015","2016", "2017",]),
                    y = Texas.n_killed,
                    mode = "lines+markers",
                    name = "",
                    marker = dict(color = 'rgba(80, 26, 80, 0.8)'),
                    text= "")

data = [trace1]
layout = dict(title = 'Number of total attack (killed and injured) in Texas between 2014 and 2017',
              xaxis= dict(title= 'Year',ticklen= 5,zeroline= False),
              yaxis= dict(title= 'Number of ppl Killed',ticklen= 5,zeroline= False)
             )
fig = dict(data = data, layout = layout)
plotly.offline.plot(fig, "killChart.html")

'file:///content/temp-plot.html'